In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains as AC
import time


In [2]:
driver=webdriver.Edge()
driver.get("https://www.imdb.com/search/title/?title_type=feature&genres=romance&release_date=2024-01-01,2024-12-31")
time.sleep(5)
movie_name=driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

def more_list():
    try:
        more_button=driver.find_element(By.XPATH,'//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button/span/span')
        AC(driver).move_to_element(more_button).perform()
        more_button.click()
        time.sleep(5)
        return True
    except:
        return False

while more_list():
    print("Click for more")

print("NO more")


Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
Click for more
NO more


In [3]:
movie_name=driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')

In [4]:
len(movie_name)

1163

In [5]:
time.sleep(3)
titles=[]
ratings=[]
votings=[]
durations=[]
movie_name=driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')
for movies in movie_name: 
    try:  
        title=movies.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
        rating=movies.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
        voting=movies.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text
        duration=movies.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text
        titles.append(title)
        ratings.append(rating)
        votings.append(voting)
        durations.append(duration)
    except:
        pass
        


In [6]:
ROM_Movies=pd.DataFrame({
    "Title":titles,
    "Rating":ratings,
    "Voting":votings,
    "Duration":durations
})

ROM_Movies


,Title,Rating,Voting,Duration
0,1. Babygirl,6.1,(36K),1h 54m
1,2. Anora,7.8,(92K),2h 19m
2,3. Wicked,7.6,(125K),2h 40m
3,4. It Ends with Us,6.4,(75K),2h 10m
4,5. We Live in Time,7.0,(41K),1h 48m
...,...,...,...,...
664,1080. Tokhon Jokhon,7.3,(11),1h 3m
665,1092. Borjatri,8.4,(6),1h 12m
666,1093. Osompurno,9.6,(12),59m
667,1152. A Lovely Résumé,9.2,(6),1h 10m


In [7]:
#ROM_Movies['Voting'] = ROM_Movies['Voting'].apply(lambda x: int(float(x.replace('K', '')) * 1000))

In [8]:
def convert_voting(value):
    value = value.strip().replace('(', '').replace(')', '')  # Remove spaces and parentheses
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)  # Convert 'K' to a number
    elif value.isdigit():  # Check if it's a pure number
        return int(value)
    else:
        return None

In [9]:
ROM_Movies["Voting"]=ROM_Movies["Voting"].apply(convert_voting)

In [10]:
#ROM_Movies["Genre"]="Romance"
#ROM_Movies

In [11]:
#ROM_Movies.to_csv("project_rom.csv", index=False)

In [12]:
def convert_to_sql_time(value):
    value = value.lower().strip()  
    if 'h' in value or 'm' in value:
        value = value.replace('h', '').replace('m', '')  
        parts = value.split() 
        hours = int(parts[0]) if len(parts) > 0 and parts[0].isdigit() else 0
        minutes = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else 0
        return f"{hours:02d}:{minutes:02d}:00"  
    return None

In [13]:
ROM_Movies["Duration"]= ROM_Movies["Duration"].apply(convert_to_sql_time)

In [18]:
ROM_Movies['Total_Duration'] = pd.to_timedelta(ROM_Movies['Duration']).dt.total_seconds() / 60

In [19]:
ROM_Movies["Genre"]="Romance"
ROM_Movies

,Title,Rating,Voting,Duration,Genre,Total_Duration
0,1. Babygirl,6.1,36000,01:54:00,Romance,114.0
1,2. Anora,7.8,92000,02:19:00,Romance,139.0
2,3. Wicked,7.6,125000,02:40:00,Romance,160.0
3,4. It Ends with Us,6.4,75000,02:10:00,Romance,130.0
4,5. We Live in Time,7.0,41000,01:48:00,Romance,108.0
...,...,...,...,...,...,...
664,1080. Tokhon Jokhon,7.3,11,01:03:00,Romance,63.0
665,1092. Borjatri,8.4,6,01:12:00,Romance,72.0
666,1093. Osompurno,9.6,12,59:00:00,Romance,3540.0
667,1152. A Lovely Résumé,9.2,6,01:10:00,Romance,70.0


In [20]:
del ROM_Movies['Total_Duration']

In [21]:
ROM_Movies

,Title,Rating,Voting,Duration,Genre
0,1. Babygirl,6.1,36000,01:54:00,Romance
1,2. Anora,7.8,92000,02:19:00,Romance
2,3. Wicked,7.6,125000,02:40:00,Romance
3,4. It Ends with Us,6.4,75000,02:10:00,Romance
4,5. We Live in Time,7.0,41000,01:48:00,Romance
...,...,...,...,...,...
664,1080. Tokhon Jokhon,7.3,11,01:03:00,Romance
665,1092. Borjatri,8.4,6,01:12:00,Romance
666,1093. Osompurno,9.6,12,59:00:00,Romance
667,1152. A Lovely Résumé,9.2,6,01:10:00,Romance


In [22]:
ROM_Movies.to_csv("project_rom.csv", index=False)